In [ ]:
inflation = {1970:6.456185567,
1971:6.185185185,
1972:5.992822967,
1973:5.641891892,
1974:5.081135903,
1975:4.656133829,
1976:4.402460457,
1977:4.133663366,
1978:3.84202454,
1979:3.450413223,
1980:3.040048544,
1981:2.755775578,
1982:2.595854922,
1983:2.515060241,
1984:2.410972089,
1985:2.328066914,
1986:2.285583942,
1987:2.205105634,
1988:2.117497887,
1989:2.02016129,
1990:1.916602907,
1991:1.839207048,
1992:1.785459729,
1993:1.733564014,
1994:1.690283401,
1995:1.643700787,
1996:1.596558317,
1997:1.560747664,
1998:1.536809816,
1999:1.503601441,
2000:1.454703833,
2001:1.41445511,
2002:1.392440245,
2003:1.361413043,
2004:1.326098465,
2005:1.282642089,
2006:1.242559524,
2007:1.208393632,
2008:1.163492801,
2009:1.167832168,
2010:1.148555708,
2011:1.113828368,
2012:1.091027875,
2013:1.075107296,
2014:1.058301648,
2015:1.056962025,
2016:1.04375,
2017:1.022031824,
2018:1}

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import fnmatch
import zipfile
import xmltodict


In [ ]:
data = []
data2 = []
for file in os.listdir('nsf'):
    if fnmatch.fnmatch(file,'*.zip'):
        year, _ = file.split('.')
        if int(year) < 1970:
            continue
        archive = zipfile.ZipFile(os.path.join('nsf',file), 'r')
        print(file)
        for zfile in archive.filelist:
            try: 
                xmlf = archive.read(zfile)
                o = xmltodict.parse(xmlf)

                n = 1
                y = o['rootTag']['Award']['AwardAmount']
                xs = []
                if 'Investigator' in o['rootTag']['Award']:
                    if 'FirstName' in o['rootTag']['Award']['Investigator']:
                        if o['rootTag']['Award']['Investigator']['FirstName'] is None:
                                continue
                        x = o['rootTag']['Award']['Investigator']['FirstName'] + ' ' + o['rootTag']['Award']['Investigator']['LastName']
                        data.append({'name' : x, 'award': y,'year':year,'n':n})
                        xs.append(x)
                    else:
                        n = len(o['rootTag']['Award']['Investigator'])
                        for inv in o['rootTag']['Award']['Investigator']:
                            if inv['FirstName'] is None:
                                continue
                            x = inv['FirstName'] + ' ' + inv['LastName']
                            data.append({'name' : x, 'award': y,'year':year,'n':n})
                            xs.append(x)
                    data2.append({'name' : xs, 'award': y,'year':year,'n':n})
            except:
                print(year)
                #print(inv)
                #print(o['rootTag']['Award']['Investigator'])
                pass


In [ ]:
df = pd.DataFrame(data)
df2 = pd.DataFrame(data2)

In [ ]:
df.shape,df2.shape
df.award = pd.to_numeric(df.award)
df.year = pd.to_numeric(df.year)
df['avg_award'] = df.award/df.n
df2.award = pd.to_numeric(df2.award)
df2.year = pd.to_numeric(df2.year)




In [ ]:
#df.to_pickle('nsf.pkl')
#df2.to_pickle('nsf2.pkl')
df2 = pd.read_pickle('nsf2.pkl')
df = pd.read_pickle('nsf.pkl')
df['infaward']= df.award * df.year.map(inflation)
df2['infaward'] = df2.award * df2.year.map(inflation)
df['avg_award'] = df.award/df.n
df['avg_infaward'] = df.infaward/df.n
df.to_pickle('nsf.pkl')
df2.to_pickle('nsf2.pkl')


In [ ]:
df[df.name == 'Abhinav Gupta']

In [ ]:
df[df.name == 'Martial Hebert']

In [ ]:
df[df.name == 'Christopher Atkeson'].infaward.astype(np.int).sum(), df[df.name == 'Martial Hebert'].infaward.astype(np.int).sum()

In [ ]:
df[df.name == 'Christopher Atkeson'].avg_infaward.sum(), df[df.name == 'Martial Hebert'].avg_infaward.sum(), df[df.name == 'Abhinav Gupta'].avg_infaward.sum()